In [1]:
import networkx as nx
import os
from scipy.io import mmread

In [18]:
def load_graph(filepath: str) -> nx.Graph:
    """
    Carrega grafos de arquivos .txt, .edges, .edge, .csv, .gml ou mtx.
    Detecta automaticamente:
    1. Delimitadores: Espaço, Vírgula (CSV) ou Tabulação.
    2. Comentários: '#' ou '%'.
    3. Cabeçalhos de texto (ex: id_1, id_2) e os remove.
    """
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Arquivo não encontrado: {filepath}")

    # Se for GML, usa o leitor específico
    if filepath.endswith('.gml'):
        return nx.read_gml(filepath, label='id')
    
    if filepath.endswith(".mtx"):
        sparse_matrix = mmread(filepath)
        G = nx.from_scipy_sparse_array(sparse_matrix)
        g = G.to_undirected()
        G.remove_edges_from(nx.selfloop_edges(G))
        return G


    # Configurações padrão
    delimiter = None 
    comment_char = '#'
    has_header = False # Nova flag para controle
    
    # 1. ESPIAR O ARQUIVO (Detecção de metadados e cabeçalho)
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if not line: continue 
            
            # Se for comentário explícito, detectamos o caractere e seguimos
            if line.startswith(('%', '#')):
                comment_char = line[0]
                continue
            
           
            # Detecta separador
            if ',' in line: delimiter = ','
            elif '\t' in line: delimiter = '\t'
            else: delimiter = None # Espaço

            # Teste de conversão: Tenta converter o primeiro elemento para int
            parts = line.split(delimiter) if delimiter else line.split()
            try:
                int(parts[0]) # Se conseguir virar número, é dado!
                has_header = False
            except ValueError:
                has_header = True
            
            break 

    print(f"Lendo {filepath} | Delim: '{'espaço' if delimiter is None else delimiter}' | Header: {has_header}")

    # 2. LER O ARQUIVO (Pulando o cabeçalho se necessário)
    try:
        with open(filepath, 'r') as f:
            if has_header:
                next(f)
            G = nx.read_edgelist(
                f, 
                nodetype=int,          
                comments=comment_char, 
                delimiter=delimiter,   
                create_using=nx.Graph(), 
                data=False             
            )
        
        G.remove_edges_from(nx.selfloop_edges(G))
        return G
        
    except Exception as e:
        raise ValueError(f"Erro crítico ao ler {filepath}: {e}")
 

In [19]:
dir_path = "../data"
arquivos_instancias = os.listdir("../data")
files = []
instancias = []
for file in arquivos_instancias:
    full_path = os.path.join(dir_path, file)
    if os.path.isfile(full_path):
        nome, base = os.path.splitext(file)
        G = load_graph(full_path)
        instancias.append(G)


            

      

    

Lendo ../data/email-Enron.txt | Delim: '	' | Header: False
Lendo ../data/ca-GrQc.txt | Delim: '	' | Header: False
Lendo ../data/ca-AstroPh.txt | Delim: '	' | Header: False
Lendo ../data/ca-HepTh.txt | Delim: '	' | Header: False
Lendo ../data/ca-HepPh.txt | Delim: '	' | Header: False
Lendo ../data/ca-CondMat.txt | Delim: '	' | Header: False
Lendo ../data/socfb-nips-ego.edges | Delim: 'espaço' | Header: False
Lendo ../data/soc-gplus.edges | Delim: 'espaço' | Header: False
Lendo ../data/musae_git_edges.csv | Delim: ',' | Header: True
Lendo ../data/loc-gowalla_edges.txt | Delim: '	' | Header: False
Lendo ../data/artist_edges.csv | Delim: ',' | Header: True
Lendo ../data/HR_edges.csv | Delim: ',' | Header: True
Lendo ../data/com-dblp.ungraph.txt | Delim: '	' | Header: False
Lendo ../data/amazon0302.txt | Delim: '	' | Header: False
Lendo ../data/amazon0312.txt | Delim: '	' | Header: False
Lendo ../data/amazon0505.txt | Delim: '	' | Header: False
Lendo ../data/amazon0601.txt | Delim: '	' | He

24